<a href="https://colab.research.google.com/github/tomonari-masada/courses/blob/master/10_document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 機械学習で文書分類を試みる

* WikipediaからUSの男性俳優と女性俳優のページをクローリングして、spaCyで簡単な前処理を済ませてあるデータを使う。

 * 固有名詞、冠詞、前置詞、代名詞、副詞、数詞、接続詞は除去してある。
 * lemmatizationした結果を使っている。
 * "actor"と"actress"という単語は特別に除去してある。

* 分析の目的1: 検証データでできるかぎりチューニングを行い、最後にテストデータでの分類性能を明らかにする。

* 分析の目的2： 男性俳優と女性俳優のページを分類する際に、どのような単語が特に効いているかを調べる。

 * この調査によって、俳優に関する記述におけるジェンダー・ステレオタイプを明らかにできるか？

In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

### 1) データファイルを読み込む

* データファイルは、あらかじめ自分のGoogle Driveの適当なフォルダに置いておく。

* データファイルの各行には、女性俳優(1)か男性俳優(0)かを表すフラグ、俳優の名前、Wikipediaのページの本文が、この順に格納されている。

* データファイルの各行に対してeval組み込み関数を適用すると、Pythonのリストに変換できるようなフォーマットで、ファイルに記録されている。

In [2]:
y = list()
names = list()
corpus = list()
with open('drive/My Drive/data/us_actors_and_actresses.txt', 'r') as f:
    for line in f:
        flag, name, text = eval(line.strip())
        y.append(int(flag))
        names.append(name)
        corpus.append(text)

In [3]:
# corpusは、本文の文字列がたくさん入っているリスト。
corpus[:10]

['bear american bond appear guest television show feature film film production company call fellow bear convict manslaughter death aunt boyfriend finish serve year sentence',
 'american film director',
 'american film bear appear film remember role several film give era perform cast stereotypical role common film time boundless energy can see restrictive role master become know scared reaction comedy appear soundie musical film viewer will remember portrayal startled cook quizzical orderly frighten porter apprehensive watchman such film hold play lovable train conductor bug eye shriek ah lion attack rip seat pant shine pair shoe gag use previous short portrayal scene funny crew could contain laughter can hear final release memorable role hapless chef plumbing Will go utter bewilderment house sho go crazy able show range act talent role able raise laugh audience give suspicious sideway look kitchen appliance act footage would recycle more future comedy include film scene raincoat clothe

In [4]:
# yは男性俳優か女性俳優かを表す0/1のresponse。
# namesは俳優の名前。
y = np.array(y)
names = np.array(names)
print(names[:20])

['De%27Aundre_Bonds' 'James_McLaughlin_(actor)' 'Dudley_Dickerson'
 'Charles_P._Thompson' 'Gus_Saville' 'Paris_Themmen' 'Peter_Haskell'
 'Robert_Keith_(actor)' 'Thomas_Carroll_(martial_artist)'
 'Matthew_Dickens' 'Nat_Benchley' 'Richard_Hunt_(puppeteer)'
 'Thomas_McDonell' 'Paul_Smith_(American_actor,_born_1929)' 'Jim_Zulevic'
 'Rob_Moran' 'Harry_Bannister' 'Shaun_Weiss' 'Peter_Sarsgaard'
 'Palmer_Williams_Jr.']


In [5]:
# Nは全文書数。
N = len(y)
print('We have {:d} documents.'.format(N))

We have 19645 documents.


### 2) テストデータを分離

In [6]:
# 訓練データとテストデータを分割するために、インデックスをランダムにシャッフルする。
indices = np.arange(N)
np.random.seed(123) # 乱数のシードを固定して再現性を持たせている。
np.random.shuffle(indices)
print(indices)

[12617 18144 17561 ... 15377 17730 15725]


In [7]:
# 手動で訓練データ8割、テストデータ2割に分割する。
border = N * 8 // 10
print('We have {:d} test documents.'.format(N - border))
train_indices = indices[:border]
test_indices = indices[border:]
y_train = y[train_indices]
y_test = y[test_indices]
names_train = names[train_indices]
names_test = names[test_indices]

We have 3929 test documents.


In [8]:
# 本文のほうも、インデックスの同じ分割を使って、訓練データとテストデータに分割する。
corpus_train = list()
for i in train_indices:
  corpus_train.append(corpus[i])
corpus_test = list()
for i in test_indices:
  corpus_test.append(corpus[i])

（ここまでのtraining setとtest setへの分割は、変えないようにしてください。）

### 3) TF-IDFで各文書をベクトル化する

* TF-IDFは単語列をベクトル化する方法のひとつ。

* ベクトルの次元は語彙数となる。各Wikipediaのページがひとつのベクトルへ変換される。

* TfidfVectorizerのパラメータをチューニングしても構わない

 * ここでTF-IDFの計算をするときにテストデータは使っていないので、ズルはしていない。

 * min_dfは、その数より少ない文書にしか出現しない単語を削除する、という意味のパラメータ。希少な単語を削除するために使う。

 * max_dfは、0から1の間の実数で指定すると、その割合より多い文書に出現する単語を削除する、という意味のパラメータ。ありふれた単語を削除するために使う。

In [9]:
vectorizer = TfidfVectorizer(stop_words='english', max_df=0.5, min_df=50)
X_train = vectorizer.fit_transform(corpus_train)
print('# X_train shape', X_train.shape)

# X_train shape (15716, 3976)


In [10]:
# 語彙を取得する。
vocab = np.array(vectorizer.get_feature_names())

In [11]:
# 語彙の一部を見てみる（アルファベット順に並んでいるようだ）。
print(vocab[1000:1010])

['define' 'defunct' 'defy' 'degree' 'delay' 'delete' 'delight' 'deliver'
 'delivery' 'demand']


### 4) 5-fold 交差検証を使った、ロジスティック回帰による２値分類と評価

* ロジスティック回帰のハイパーパラメータは、検証データでの分類性能ができるだけよくなるように、変更する。

In [12]:
from sklearn.model_selection import StratifiedKFold

In [13]:
# これは単純な実行例にすぎません。正則化も含めてチューニングしてください。
# TfidfVectorizerのパラメータも併せてチューニングしていいです。

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
for train_index, valid_index in skf.split(X_train, y_train):
  clf = LogisticRegression(max_iter=1000, solver='liblinear', random_state=123)
  clf.fit(X_train[train_index], y_train[train_index])
  print('validation mean accuracy: {}'.format(clf.score(X_train[valid_index], y_train[valid_index])))

validation mean accuracy: 0.8396946564885496
validation mean accuracy: 0.8409163219853643
validation mean accuracy: 0.8390073178491887
validation mean accuracy: 0.8320076360165447
validation mean accuracy: 0.8304167992363983


### 5) チューニングされたハイパーパラメータを使って、ロジスティック回帰の最終的な評価をおこなう

* LogisticRegression() のカッコ内には、自分で見つけた最善のセッティングを書き込む。
* 学習は、訓練データ全体（テストデータ以外の全体）を使っている。

In [14]:
# 最も良かった設定を使って、訓練データ全体で再訓練
clf = LogisticRegression(max_iter=1000, solver='liblinear', random_state=123)
clf.fit(X_train, y_train)

# そして、最終的にテストデータで評価
X_test = vectorizer.transform(corpus_test)
print('test mean accuracy: {}'.format(clf.score(X_test, y_test)))

test mean accuracy: 0.8274370068719776


### 6) 5-fold 交差検証を使った、SVMによる２値分類と評価

In [15]:
# これは単純な実行例にすぎません。ハイパーパラメータのチューニングもしてください。

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
for train_index, valid_index in skf.split(X_train, y_train):
  svm = LinearSVC(C=0.1, max_iter=2000, random_state=123)
  svm.fit(X_train[train_index], y_train[train_index])
  print('validation mean accuracy: {}'.format(svm.score(X_train[valid_index], y_train[valid_index])))

validation mean accuracy: 0.8400127226463104
validation mean accuracy: 0.8428253261215399
validation mean accuracy: 0.8351893095768375
validation mean accuracy: 0.8320076360165447
validation mean accuracy: 0.8313713013044861


### 7) チューニングされたハイパーパラメータを使って、SVMの最終的な評価をおこなう

In [16]:
# 最も良かった設定を使って、訓練データ全体で再訓練
svm = LinearSVC(C=0.1, max_iter=2000, random_state=123)
svm.fit(X_train, y_train)

# そして、最終的にテストデータで評価
X_test = vectorizer.transform(corpus_test)
print('test mean accuracy: {}'.format(svm.score(X_test, y_test)))

test mean accuracy: 0.8271824891829982


### 8) 分類に効いている単語を調べる

* 訓練データが最も数が多いので、訓練データの分類に最も効いている単語100語を調べる。

* 下に示すのは、あくまで一つの方法にすぎない。他にどんな方法があるか調べて使ってみよう。

 * 下の手法の欠点は、男性俳優の文書に特徴的な単語と、女性俳優の文書に特徴的な単語とを、区別できない点である。

 * ヒント： 「svm important features」 あたりでググってみる。

In [17]:
# sklearnにあるrecursive feature eliminationという特徴量選択の手法を使ってみる。
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

from sklearn.feature_selection import RFE

rfe = RFE(estimator=clf, n_features_to_select=100, step=100)
rfe.fit(X_train, y_train)
ranking = rfe.ranking_
print(vocab[ranking == 1])

['alumna' 'anime' 'announcer' 'attack' 'aunt' 'ballet' 'bar' 'baseball'
 'basketball' 'beauty' 'birth' 'blonde' 'boxing' 'boy' 'breast' 'brother'
 'cabaret' 'cheerleader' 'contest' 'cop' 'cover' 'cum' 'dancer' 'daughter'
 'debut' 'detective' 'die' 'direct' 'director' 'discharge' 'draft'
 'driver' 'enlist' 'father' 'female' 'fight' 'following' 'football'
 'fraternity' 'gang' 'gangster' 'gay' 'girl' 'girlfriend' 'granddaughter'
 'grandmother' 'guy' 'heroine' 'housekeeper' 'husband' 'joke' 'killer'
 'lady' 'lesbian' 'magazine' 'maid' 'male' 'man' 'married' 'marry'
 'matriarch' 'military' 'model' 'modeling' 'mom' 'mother' 'nephew' 'niece'
 'nurse' 'née' 'officer' 'pageant' 'positive' 'pregnant' 'prostitute'
 'queen' 'race' 'rapper' 'receptionist' 'regular' 'relocate' 'retire'
 'screenplay' 'secretary' 'serve' 'singer' 'sister' 'soldier' 'son'
 'soprano' 'sorority' 'stand' 'stuntman' 'team' 'villain' 'waiter'
 'waitress' 'wife' 'witch' 'woman']


In [18]:
rfe = RFE(estimator=svm, n_features_to_select=100, step=100)
rfe.fit(X_train, y_train)
ranking = rfe.ranking_
print(vocab[ranking == 1])

['acting' 'alumna' 'anchor' 'anime' 'announcer' 'attack' 'aunt' 'ballet'
 'band' 'bar' 'baritone' 'baseball' 'basketball' 'beauty' 'birth' 'blonde'
 'boy' 'breast' 'brother' 'cabaret' 'cheerleader' 'contest' 'cop' 'cover'
 'dancer' 'daughter' 'debut' 'detective' 'direct' 'director' 'discharge'
 'draft' 'driver' 'enlist' 'father' 'female' 'fight' 'following'
 'football' 'fraternity' 'gang' 'gay' 'girl' 'girlfriend' 'granddaughter'
 'grandmother' 'guitar' 'guy' 'heroine' 'housekeeper' 'husband' 'joke'
 'killer' 'lady' 'lesbian' 'magazine' 'maid' 'male' 'man' 'married'
 'marry' 'matriarch' 'military' 'model' 'modeling' 'mom' 'mother' 'nephew'
 'niece' 'nurse' 'née' 'officer' 'pageant' 'pregnant' 'prostitute' 'queen'
 'race' 'rapper' 'receptionist' 'relocate' 'retire' 'screenplay'
 'secretary' 'serve' 'singer' 'sister' 'soldier' 'son' 'soprano'
 'sorority' 'spokesman' 'stand' 'stuntman' 'team' 'villain' 'waiter'
 'waitress' 'wife' 'witch' 'woman']
